# BANK ACCOUNT

In [10]:
from datetime import datetime, timedelta
from decimal import Decimal
import json

class BankAccount:
    def __init__(self):
        self.accounts = {}
        self.loans = {}
        self.fixed_deposits = {}
        self.interest_rates = {
            "savings": Decimal("0.04"),    # 4% annual
            "current": Decimal("0.01"),    # 1% annual
            "loan": {
                "personal": Decimal("0.12"),    # 12% annual
                "home": Decimal("0.08"),        # 8% annual
                "vehicle": Decimal("0.10")      # 10% annual
            },
            "fd": {
                "6_months": Decimal("0.055"),   # 5.5% annual
                "1_year": Decimal("0.065"),     # 6.5% annual
                "2_years": Decimal("0.070")     # 7.0% annual
            }
        }

    def create_account(self, account_number, primary_holder, age, address, account_type, 
                      pin, initial_deposit=0, joint_holder=None, email=None, mobile=None):
        """Create a new bank account with optional joint holder"""
        if account_number in self.accounts:
            return "Account number already exists"
            
        if initial_deposit < 0:
            return "Initial deposit cannot be negative"
            
        if account_type not in ["savings", "current"]:
            return "Invalid account type. Choose 'savings' or 'current'"
            
        if len(str(pin)) != 4:
            return "PIN must be 4 digits"
            
        account = {
            "primary_holder": primary_holder,
            "joint_holder": joint_holder,
            "age": age,
            "address": address,
            "account_type": account_type,
            "balance": Decimal(str(initial_deposit)),
            "pin": pin,
            "transactions": [],
            "creation_date": datetime.now(),
            "last_interest_calculation": datetime.now(),
            "email": email,
            "mobile": mobile,
            "notifications": []
        }
        
        self.accounts[account_number] = account
        self._add_transaction(account_number, "credit", initial_deposit, "Initial deposit")
        self._send_notification(account_number, "Account created successfully")
        return f"Account created successfully for {primary_holder}"

    def create_fixed_deposit(self, account_number, amount, duration, pin):
        """Create a fixed deposit linked to an existing account"""
        if account_number not in self.accounts:
            return "Account not found"
            
        if not self._verify_pin(account_number, pin):
            return "Invalid PIN"
            
        if amount <= 0:
            return "Deposit amount must be positive"
            
        if self.accounts[account_number]["balance"] < amount:
            return "Insufficient balance in primary account"
            
        fd_id = f"FD_{account_number}_{len(self.fixed_deposits)}"
        
        # Convert duration to standard period
        if duration not in ["6_months", "1_year", "2_years"]:
            return "Invalid duration. Choose '6_months', '1_year', or '2_years'"
            
        interest_rate = self.interest_rates["fd"][duration]
        maturity_date = datetime.now() + timedelta(days=180 if duration == "6_months" else 365 if duration == "1_year" else 730)
        
        fd = {
            "account_number": account_number,
            "amount": Decimal(str(amount)),
            "interest_rate": interest_rate,
            "start_date": datetime.now(),
            "maturity_date": maturity_date,
            "duration": duration,
            "status": "active"
        }
        
        self.fixed_deposits[fd_id] = fd
        self.accounts[account_number]["balance"] -= amount
        self._add_transaction(account_number, "debit", amount, f"Fixed Deposit creation - {fd_id}")
        self._send_notification(account_number, f"Fixed Deposit {fd_id} created successfully")
        
        return f"Fixed Deposit created with ID: {fd_id}"

    def apply_for_loan(self, account_number, loan_type, amount, duration_months, pin):
        """Apply for a loan"""
        if account_number not in self.accounts:
            return "Account not found"
            
        if not self._verify_pin(account_number, pin):
            return "Invalid PIN"
            
        if loan_type not in ["personal", "home", "vehicle"]:
            return "Invalid loan type"
            
        # Basic eligibility check
        account = self.accounts[account_number]
        if account["age"] < 21:
            return "Minimum age requirement not met"
            
        loan_id = f"LOAN_{account_number}_{len(self.loans)}"
        interest_rate = self.interest_rates["loan"][loan_type]
        
        # Calculate EMI: EMI = P * R * (1 + R)^N / ((1 + R)^N - 1)
        # where P = Principal, R = Monthly interest rate, N = Number of months
        monthly_rate = interest_rate / 12
        emi = (amount * monthly_rate * (1 + monthly_rate) ** duration_months) / ((1 + monthly_rate) ** duration_months - 1)
        
        loan = {
            "account_number": account_number,
            "loan_type": loan_type,
            "amount": Decimal(str(amount)),
            "interest_rate": interest_rate,
            "duration_months": duration_months,
            "start_date": datetime.now(),
            "end_date": datetime.now() + timedelta(days=30*duration_months),
            "emi": Decimal(str(emi)),
            "remaining_amount": Decimal(str(amount)),
            "status": "active",
            "payments": []
        }
        
        self.loans[loan_id] = loan
        self.accounts[account_number]["balance"] += amount
        self._add_transaction(account_number, "credit", amount, f"Loan disbursement - {loan_id}")
        self._send_notification(account_number, f"Loan {loan_id} approved and disbursed")
        
        return f"""
Loan approved!
Loan ID: {loan_id}
Amount: {amount}
Duration: {duration_months} months
EMI: {emi:.2f}
Interest Rate: {interest_rate*100}% per annum
"""

    def pay_loan_emi(self, loan_id, account_number, pin):
        """Pay EMI for a loan"""
        if loan_id not in self.loans:
            return "Loan not found"
            
        if account_number not in self.accounts:
            return "Account not found"
            
        if not self._verify_pin(account_number, pin):
            return "Invalid PIN"
            
        loan = self.loans[loan_id]
        if loan["account_number"] != account_number:
            return "Loan not associated with this account"
            
        if loan["status"] != "active":
            return "Loan is not active"
            
        if self.accounts[account_number]["balance"] < loan["emi"]:
            return "Insufficient balance for EMI payment"
            
        self.accounts[account_number]["balance"] -= loan["emi"]
        loan["remaining_amount"] -= loan["emi"]
        
        payment = {
            "date": datetime.now(),
            "amount": loan["emi"]
        }
        loan["payments"].append(payment)
        
        if loan["remaining_amount"] <= 0:
            loan["status"] = "completed"
            self._send_notification(account_number, f"Loan {loan_id} has been fully repaid")
        
        self._add_transaction(account_number, "debit", loan["emi"], f"Loan EMI payment - {loan_id}")
        return f"EMI payment successful. Remaining loan amount: {loan['remaining_amount']:.2f}"

    def _send_notification(self, account_number, message):
        """Send notification via email/mobile (simulated)"""
        account = self.accounts[account_number]
        notification = {
            "timestamp": datetime.now(),
            "message": message
        }
        account["notifications"].append(notification)
        
        # In a real system, you would integrate with email/SMS services here
        if account["email"]:
            print(f"Email sent to {account['email']}: {message}")
        if account["mobile"]:
            print(f"SMS sent to {account['mobile']}: {message}")

    def get_notifications(self, account_number, pin):
        """Get all notifications for an account"""
        if account_number not in self.accounts:
            return "Account not found"
            
        if not self._verify_pin(account_number, pin):
            return "Invalid PIN"
            
        return self.accounts[account_number]["notifications"]

    # ... (previous methods remain the same) ...

# Example usage
#if __name__ == "__main__":
#    bank = BankAccount()
    
    # Create a joint account
#    print(bank.create_account(
 #       "1001", "rafay", 25, "123 Main St, karachi",
 #       "savings", 1234, 10000,
  #      joint_holder="munawar",
   #     email="rafay@example.com",
    #    mobile="923343543510"
    #))
    
    # Create a fixed deposit
    #print(bank.create_fixed_deposit("1001", 5000, "1_year", 1234))
    
    # Apply for a loan
   # print(bank.apply_for_loan("1001", "personal", 100000, 24, 1234))
    
    # Pay loan EMI
    #print(bank.pay_loan_emi("LOAN_1001_0", "1001", 1234))
    
    # Check notifications
    #print(bank.get_notifications("1001", 1234))

In [ ]:
from datetime import datetime, timedelta
from decimal import Decimal
import random

class BankingSystem:
    def __init__(self):
        self.accounts = {}
        self.loans = {}
        self.fixed_deposits = {}
        self.last_account_number = 1000  # Starting account number
        
        self.interest_rates = {
            "savings": Decimal("0.04"),    # 4% annual
            "current": Decimal("0.01"),    # 1% annual
            "loan": {
                "personal": Decimal("0.12"),
                "home": Decimal("0.08"),
                "vehicle": Decimal("0.10")
            },
            "fd": {
                "6_months": Decimal("0.055"),
                "1_year": Decimal("0.065"),
                "2_years": Decimal("0.070")
            }
        }

    def generate_account_number(self):
        """Generate a unique account number"""
        self.last_account_number += 1
        return f"ACC{self.last_account_number}"

    def open_account(self):
        """Interactive account opening process"""
        print("\n=== Account Opening Form ===")
        name = input("Enter your full name: ")
        age = int(input("Enter your age: "))
        address = input("Enter your address: ")
        email = input("Enter your email: ")
        mobile = input("Enter your mobile number: ")
        
        print("\nSelect Account Type:")
        print("1. Savings")
        print("2. Current")
        account_type = input("Enter choice (1/2): ")
        account_type = "savings" if account_type == "1" else "current"
        
        initial_deposit = Decimal(input("\nEnter initial deposit amount (minimum 1000RS): "))
        if initial_deposit < 1000:
            print("Minimum deposit of 1000RS is required")
            return
        
        pin = input("Set your 4-digit PIN: ")
        if not (pin.isdigit() and len(pin) == 4):
            print("PIN must be 4 digits")
            return
        
        joint_holder = input("Enter joint holder name (press Enter if none): ").strip() or None
        
        account_number = self.generate_account_number()
        
        account = {
            "account_number": account_number,
            "primary_holder": name,
            "joint_holder": joint_holder,
            "age": age,
            "address": address,
            "account_type": account_type,
            "balance": initial_deposit,
            "pin": pin,
            "email": email,
            "mobile": mobile,
            "transactions": [],
            "creation_date": datetime.now(),
            "notifications": []
        }
        
        self.accounts[account_number] = account
        print(f"\nCongratulations! Your account has been created successfully.")
        print(f"Your Account Number is: {account_number}")
        print("Please save this account number for future transactions.")
        return account_number

    def login(self):
        """Login to account"""
        account_number = input("\nEnter your account number: ")
        pin = input("Enter your PIN: ")
        
        if account_number not in self.accounts:
            print("Account not found")
            return None
        
        if self.accounts[account_number]["pin"] != pin:
            print("Invalid PIN")
            return None
            
        return account_number

    def show_menu(self):
        """Display main menu"""
        print("\n=== Banking System Menu ===")
        print("1. Open New Account")
        print("2. Login to Existing Account")
        print("3. Exit")
        return input("Enter your choice (1-3): ")

    def show_account_menu(self):
        """Display account operations menu"""
        print("\n=== Account Operations ===")
        print("1. Check Balance")
        print("2. Deposit")
        print("3. Withdraw")
        print("4. Create Fixed Deposit")
        print("5. Apply for Loan")
        print("6. Pay Loan EMI")
        print("7. View Statement")
        print("8. View Notifications")
        print("9. Logout")
        return input("Enter your choice (1-9): ")

    def run(self):
        """Main program loop"""
        while True:
            choice = self.show_menu()
            
            if choice == "1":
                self.open_account()
            
            elif choice == "2":
                account_number = self.login()
                if account_number:
                    self.account_operations(account_number)
            
            elif choice == "3":
                print("Thank you for using our banking system!")
                break
            
            else:
                print("Invalid choice. Please try again.")

    def account_operations(self, account_number):
        """Handle account operations"""
        while True:
            choice = self.show_account_menu()
            
            if choice == "1":  # Check Balance
                balance = self.accounts[account_number]["balance"]
                print(f"\nCurrent Balance: PKR{balance:,.2f}")
                
            elif choice == "2":  # Deposit
                amount = Decimal(input("Enter amount to deposit: "))
                if amount > 0:
                    self.accounts[account_number]["balance"] += amount
                    print(f"Deposited PKR{amount:,.2f}")
                    print(f"New Balance: PKR{self.accounts[account_number]['balance']:,.2f}")
                
            elif choice == "3":  # Withdraw
                amount = Decimal(input("Enter amount to withdraw: "))
                if amount > self.accounts[account_number]["balance"]:
                    print("Insufficient balance")
                else:
                    self.accounts[account_number]["balance"] -= amount
                    print(f"Withdrawn PKR{amount:,.2f}")
                    print(f"New Balance: PKR{self.accounts[account_number]['balance']:,.2f}")
                
            elif choice == "4":  # Create Fixed Deposit
                amount = Decimal(input("Enter FD amount: "))
                print("\nSelect Duration:")
                print("1. 6 months")
                print("2. 1 year")
                print("3. 2 years")
                duration_choice = input("Enter choice (1-3): ")
                duration_map = {"1": "6_months", "2": "1_year", "3": "2_years"}
                duration = duration_map.get(duration_choice)
                
                if duration and amount > 0:
                    if self.accounts[account_number]["balance"] >= amount:
                        fd_id = f"FD_{account_number}_{len(self.fixed_deposits)}"
                        self.accounts[account_number]["balance"] -= amount
                        self.fixed_deposits[fd_id] = {
                            "amount": amount,
                            "duration": duration,
                            "interest_rate": self.interest_rates["fd"][duration],
                            "start_date": datetime.now(),
                            "maturity_date": datetime.now() + timedelta(days=180 if duration == "6_months" else 365 if duration == "1_year" else 730)
                        }
                        print(f"Fixed Deposit created successfully. FD ID: {fd_id}")
                    else:
                        print("Insufficient balance")
                
            elif choice == "5":  # Apply for Loan
                print("\nSelect Loan Type:")
                print("1. Personal Loan")
                print("2. Home Loan")
                print("3. Vehicle Loan")
                loan_choice = input("Enter choice (1-3): ")
                loan_types = {"1": "personal", "2": "home", "3": "vehicle"}
                loan_type = loan_types.get(loan_choice)
                
                if loan_type:
                    amount = Decimal(input("Enter loan amount: "))
                    duration = int(input("Enter loan duration in months: "))
                    
                    loan_id = f"LOAN_{account_number}_{len(self.loans)}"
                    interest_rate = self.interest_rates["loan"][loan_type]
                    monthly_rate = interest_rate / 12
                    emi = (amount * monthly_rate * (1 + monthly_rate) ** duration) / ((1 + monthly_rate) ** duration - 1)
                    
                    self.loans[loan_id] = {
                        "amount": amount,
                        "duration": duration,
                        "interest_rate": interest_rate,
                        "emi": emi,
                        "remaining_amount": amount
                    }
                    self.accounts[account_number]["balance"] += amount
                    print(f"\nLoan approved! Loan ID: {loan_id}")
                    print(f"EMI: PKR{emi:,.2f} per month")
                
            elif choice == "6":  # Pay Loan EMI
                loan_id = input("Enter Loan ID: ")
                if loan_id in self.loans:
                    loan = self.loans[loan_id]
                    if self.accounts[account_number]["balance"] >= loan["emi"]:
                        self.accounts[account_number]["balance"] -= loan["emi"]
                        loan["remaining_amount"] -= loan["emi"]
                        print(f"EMI paid successfully")
                        print(f"Remaining loan amount: PKR{loan['remaining_amount']:,.2f}")
                    else:
                        print("Insufficient balance for EMI payment")
                else:
                    print("Invalid Loan ID")
                
            elif choice == "7":  # View Statement
                print("\n=== Account Statement ===")
                print(f"Account Holder: {self.accounts[account_number]['primary_holder']}")
                print(f"Account Number: {account_number}")
                print(f"Current Balance: PKR{self.accounts[account_number]['balance']:,.2f}")
                
            elif choice == "8":  # View Notifications
                print("\n=== Notifications ===")
                for notification in self.accounts[account_number]["notifications"]:
                    print(f"{notification['timestamp']}: {notification['message']}")
                
            elif choice == "9":  # Logout
                print("Logged out successfully")
                break
            
            else:
                print("Invalid choice. Please try again.")

# Run the banking system
if __name__ == "__main__":
    bank = BankingSystem()
    bank.run()


=== Banking System Menu ===
1. Open New Account
2. Login to Existing Account
3. Exit


Enter your choice (1-3):  1



=== Account Opening Form ===


Enter your full name:  muhammad rafay shahzad
Enter your age:  18
Enter your address:  a/355 ahsanabad sec 4 karachi
Enter your email:  muhammadhassnan85@gmail.com
Enter your mobile number:  03343543510



Select Account Type:
1. Savings
2. Current


...


Select Loan Type:
1. Personal Loan
2. Home Loan
3. Vehicle Loan


Enter choice (1-3):  1
Enter loan amount:  30000
Enter loan duration in months:  5



Loan approved! Loan ID: LOAN_ACC1001_0
EMI: ₹6,181.19 per month

=== Account Operations ===
1. Check Balance
2. Deposit
3. Withdraw
4. Create Fixed Deposit
5. Apply for Loan
6. Pay Loan EMI
7. View Statement
8. View Notifications
9. Logout


Enter your choice (1-9):  1



Current Balance: ₹33,000.00

=== Account Operations ===
1. Check Balance
2. Deposit
3. Withdraw
4. Create Fixed Deposit
5. Apply for Loan
6. Pay Loan EMI
7. View Statement
8. View Notifications
9. Logout
